In [ ]:
!nvidia-smi

Thu Sep 24 14:08:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow.python.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.python.keras import Model
import imageio
import pandas as pd
import numpy as np
import IPython.display as display
import os
import pickle

In [ ]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = np.stack((x_train,)*3, axis=-1)
x_test = np.stack((x_test,)*3, axis=-1)

11493376/11490434 [==============================] - 0s 0us/step


In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(60000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

#test_ds

In [ ]:
class BaselineModel(Model):
  def __init__(self):
    super(BaselineModel, self).__init__()
    
    self.normalise = lambda x: (tf.cast(x, tf.float32)) / 255.0 - 1.0
    self.conv1 = Conv2D(64, 5, activation='relu')
    self.conv2 = Conv2D(128, 5, activation='relu')
    self.maxpool = MaxPool2D(2)
    self.flatten = Flatten()
    
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')
    

  def call(self, x):
    x = self.normalise(x)
    x = self.conv1(x)
    x = self.conv2(x)
    x = self.maxpool(x)
    x = self.flatten(x)
    x = self.d1(x)

    return self.d2(x)

model = BaselineModel()

In [ ]:
input_shape = (None, 28, 28, 3)
model.build(input_shape)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

Model: "baseline_model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            multiple                  4864      
_________________________________________________________________
conv2d_7 (Conv2D)            multiple                  204928    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten_3 (Flatten)          multiple                  0         
_________________________________________________________________
dense_6 (Dense)              multiple                  1638528   
_________________________________________________________________
dense_7 (Dense)              multiple                  1290      
Total params: 1,849,610
Trainable params: 1,849,610
Non-trainable params: 0
________________________________________

In [ ]:
model.fit(x_train, y_train, batch_size=4, epochs=10)

Epoch 1/10


TypeError: ignored

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [ ]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [ ]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)


In [ ]:
def reset_metrics():
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()

In [ ]:
EPOCHS = 10

for epoch in range(EPOCHS):
    reset_metrics()
    for images, labels in train_ds:
        train_step(images, labels)

    for test_data in test_ds:
        test_step(test_data[0], test_data[1])
    
    template = 'Epoch {}, Train Accuracy: {}, Source Test Accuracy: {}'
    print (template.format(epoch+1, train_accuracy.result()*100, test_accuracy.result()*100))


Epoch 1, Train Accuracy: 95.82499694824219, Source Test Accuracy: 98.29000091552734
Epoch 2, Train Accuracy: 98.71833038330078, Source Test Accuracy: 98.43999481201172
Epoch 3, Train Accuracy: 99.09166717529297, Source Test Accuracy: 98.9000015258789
Epoch 4, Train Accuracy: 99.3499984741211, Source Test Accuracy: 98.97000122070312
Epoch 5, Train Accuracy: 99.4383316040039, Source Test Accuracy: 98.98999786376953
Epoch 6, Train Accuracy: 99.53166961669922, Source Test Accuracy: 98.97999572753906
Epoch 7, Train Accuracy: 99.59666442871094, Source Test Accuracy: 98.77999877929688
Epoch 8, Train Accuracy: 99.7316665649414, Source Test Accuracy: 99.0999984741211
Epoch 9, Train Accuracy: 99.68000030517578, Source Test Accuracy: 99.12999725341797
Epoch 10, Train Accuracy: 99.69166564941406, Source Test Accuracy: 99.11000061035156
